Hii , SO here we are Fine tuning the base 4-bit quantized model - unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit

In [ ]:
!pip install unsloth

In [ ]:


!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:


from unsloth import FastLanguageModel
import torch
from peft import PeftModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map = "auto",
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

from datasets import load_dataset

dataset = load_dataset("taskydata/baize_chatbot")

print(dataset.column_names)

print(dataset['train'][0])

In [ ]:
print("First 5 examples of the subsetted dataset:")
for i in range(min(50, len(dataset["train"]))):
    print(f"Example {i}: {dataset['train'][i]}")

In [ ]:
dataset['train'][0]

In [ ]:
#adjust the dataset to match the chat template or change the chat template to match the dataset

chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:     
{INPUT}

### Response:
{OUTPUT}"""

def format_dataset_example(example):
    """Formats a single example from the dataset using the chat template."""
   
    parts = example['input'].split('[|AI|]', 1) 
    if len(parts) > 1:
        human_part = parts[0].replace('The conversation between human and AI assistant.\n', '').strip()
       
        if human_part.endswith('[|Human|]'):
             human_part = human_part[:-len('[|Human|]')].strip()

        
        ai_part = parts[1].strip()

        
        formatted_text = chat_template.format(INPUT=human_part, OUTPUT=ai_part)
        example['text'] = formatted_text
    else:
    
        example['text'] = ""
    

    return example

dataset["train"] = dataset["train"].map(format_dataset_example, num_proc=2, remove_columns=['topic', 'input'])


print("First formatted example:")
print(dataset['train'][0]['text'])

print("Dataset column names after formatting:")
print(dataset.column_names)

In [ ]:
!pip install tensorboard accelerate matplotlib

In [ ]:
from transformers.integrations import TensorBoardCallback
import matplotlib.pyplot as plt
%load_ext tensorboard

In [ ]:
import os
import datetime


timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_name_for_log = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit" 

output_dir = "outputs"
tensorboard_log_dir = os.path.join(output_dir, "runs", f"{timestamp}_{model_name_for_log}")
checkpoint_dir = os.path.join(output_dir, "checkpoints")


os.makedirs(tensorboard_log_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)


print(f"TensorBoard log directory created: {tensorboard_log_dir}")
print(f"Checkpoint directory created: {checkpoint_dir}")

In [ ]:
%tensorboard --logdir {tensorboard_log_dir}

In [ ]:
# Use the full train split for training
print(f"Using {len(dataset['train'])} examples for training.")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import math

#Hey these hyparams are for large dataset, kindly adjust according to your storage and Ram.
per_device_batch_size = 2 # Increase if your VRAM allows
grad_accum            = 16 # Increase for a larger effective batch size (32)
num_epochs            = 1  # Train for 1 or 2 epoch

max_steps = math.ceil((len(dataset["train"]) / (per_device_batch_size * grad_accum)) * num_epochs)

trainer = SFTTrainer(
    model            = model,
    tokenizer        = tokenizer,
    train_dataset    = dataset["train"],
    # eval_dataset   = dataset["test"], # Add this for evaluation
    # formatting_func= formatting_func,
    max_seq_length   = 2048, # Increased for chat data, adjust based on VRAM
    packing          = True, # and yes Set to False if needed
    args = TrainingArguments(
        per_device_train_batch_size = per_device_batch_size,
        gradient_accumulation_steps = grad_accum,
        warmup_ratio                = 0.05, 
        max_steps                   = max_steps,
        learning_rate               = 2e-5, 
        fp16                        = not is_bfloat16_supported(),
        bf16                        = is_bfloat16_supported(),
        logging_steps               = 25, # Log progress more reasonably
        optim                       = "adamw_8bit",
        weight_decay                = 0.01,
        lr_scheduler_type           = "cosine", 
        seed                        = 3407,
        output_dir                  = "outputs",
        report_to                   = "tensorboard",
        save_strategy               = "steps",
        save_steps                  = 500, # Save checkpoints periodically
        # evaluation_strategy       = "steps", # Enable evaluation
        # eval_steps                = 500,     # Evaluate every 500 steps
    ),
)

In [ ]:

trainer.args.num_train_epochs = 1
trainer_stats = trainer.train()

In [ ]:
from unsloth.chat_templates import get_chat_template


question = "Hii How are You ?."

formatted_input = chat_template.format(INPUT=question, OUTPUT="")

inputs = tokenizer(
    [formatted_input],
    return_tensors = "pt",
    add_special_tokens = False, 
).to("cuda")


outputs = model.generate(
    **inputs,
    max_new_tokens = 35, 
    use_cache = True,
    eos_token_id = tokenizer.encode("\n", add_special_tokens=True)[0], 
)


response_ids = outputs[0][inputs["input_ids"].shape[1]:]
response = tokenizer.decode(response_ids, skip_special_tokens=True)


print("Model Response:")
print(response)

After this Push your weights on Your model repo Using HuggingFAce login as see huggingface notebook..................